# NLP 2 : Neural Embeddings, Text Classification, Text Generation


To use statistical classifiers with text, it is first necessary to vectorize the text. In the first practical session we explored the **bag of word** model. 

Modern **state of the art** methods uses  embeddings to vectorize the text before classification in order to avoid feature engineering.

## Dataset
https://github.com/cedias/practicalNLP/tree/master/dataset

## "Modern" NLP pipeline

By opposition to the **bag of word** model, in the modern NLP pipeline everything is **embeddings**. Instead of encoding a text as a **sparse vector** of length $D$ (size of feature dictionnary) the goal is to encode the text in a meaningful dense vector of a small size $|e| <<< |D|$. 


The raw classification pipeline is then the following:

```
raw text ---|embedding table|-->  vectors --|Neural Net|--> class 
```


### Using a  language model:

How to tokenize the text and extract a feature dictionnary is still a manual task. To directly have meaningful embeddings, it is common to use a pre-trained language model such as `word2vec` which we explore in this practical.

In this setting, the pipeline becomes the following:
```
      
raw text ---|(pre-trained) Language Model|--> vectors --|classifier (or fine-tuning)|--> class 
```


- #### Classic word embeddings

 - [Word2Vec](https://arxiv.org/abs/1301.3781)
 - [Glove](https://nlp.stanford.edu/projects/glove/)


- #### bleeding edge language models techniques (only here for reference)

 - [UMLFIT](https://arxiv.org/abs/1801.06146)
 - [ELMO](https://arxiv.org/abs/1802.05365)
 - [GPT](https://blog.openai.com/language-unsupervised/)
 - [BERT](https://arxiv.org/abs/1810.04805)






### Goal of this session:

1. Train word embeddings on training dataset
2. Tinker with the learnt embeddings and see learnt relations
3. Tinker with pre-trained embeddings.
4. Use those embeddings for classification
5. Compare different embedding models
6. Pytorch first look: learn to generate text.





##  Loading data (same as in nlp 1)

In [18]:
import json
from collections import Counter
import sys, os, warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

#### /!\ YOU NEED TO UNZIP dataset/json_pol.zip first /!\


# Loading json
with open("data/json_pol",encoding="utf-8") as f:
    data = f.readlines()
    json_data = json.loads(data[0])
    train = json_data["train"]
    test = json_data["test"]
    

# Quick Check
counter_train = Counter((x[1] for x in train))
counter_test = Counter((x[1] for x in test))
print("Number of train reviews : ", len(train))
print("----> # of positive : ", counter_train[1])
print("----> # of negative : ", counter_train[0])
print("")
print(train[0])
print("")
print("Number of test reviews : ",len(test))
print("----> # of positive : ", counter_test[1])
print("----> # of negative : ", counter_test[0])

print("")
print(test[0])
print("")



Number of train reviews :  25000
----> # of positive :  12500
----> # of negative :  12500

["The undoubted highlight of this movie is Peter O'Toole's performance. In turn wildly comical and terribly terribly tragic. Does anybody do it better than O'Toole? I don't think so. What a great face that man has!<br /><br />The story is an odd one and quite disturbing and emotionally intense in parts (especially toward the end) but it is also oddly touching and does succeed on many levels. However, I felt the film basically revolved around Peter O'Toole's luminous performance and I'm sure I wouldn't have enjoyed it even half as much if he hadn't been in it.", 1]

Number of test reviews :  25000
----> # of positive :  12500
----> # of negative :  12500

['Although credit should have been given to Dr. Seuess for stealing the story-line of "Horton Hatches The Egg", this was a fine film. It touched both the emotions and the intellect. Due especially to the incredible performance of seven year old 

## Word2Vec: Quick Recap

**[Word2Vec](https://arxiv.org/abs/1301.3781) is composed of two distinct language models (CBOW and SG), optimized to quickly learn word vectors**


given a random text: `i'm taking the dog out for a walk`



### (a) Continuous Bag of Word (CBOW)
    -  predicts a word given a context
    
maximizing `p(dog | i'm taking the ___ out for a walk)`
    
### (b) Skip-Gram (SG)               
    -  predicts a context given a word
    
 maximizing `p(i'm taking the out for a walk | dog)`



   

## Step 1: train (or load) a language model (word2vec)

Gensim has one of [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) fastest implementation.


### Train:

In [19]:
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

text = [t.split() for t,p in train]

# the following configuration is the default configuration
w2v = gensim.models.word2vec.Word2Vec(sentences=text,
                                size=100, window=5,               ### here we train a cbow model 
                                min_count=5,                      
                                sample=0.001, workers=3,
                                sg=1, hs=0, negative=5,        ### set sg to 1 to train a sg model
                                cbow_mean=1,
                                iter=5)

2019-05-31 01:41:20,736 : INFO : collecting all words and their counts
2019-05-31 01:41:20,737 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-31 01:41:21,573 : INFO : PROGRESS: at sentence #10000, processed 2358544 words, keeping 155393 word types
2019-05-31 01:41:22,629 : INFO : PROGRESS: at sentence #20000, processed 4675912 words, keeping 243050 word types
2019-05-31 01:41:23,025 : INFO : collected 280617 word types from a corpus of 5844680 raw words and 25000 sentences
2019-05-31 01:41:23,026 : INFO : Loading a fresh vocabulary
2019-05-31 01:41:23,606 : INFO : min_count=5 retains 49345 unique words (17% of original 280617, drops 231272)
2019-05-31 01:41:23,607 : INFO : min_count=5 leaves 5517507 word corpus (94% of original 5844680, drops 327173)
2019-05-31 01:41:23,830 : INFO : deleting the raw counts dictionary of 280617 items
2019-05-31 01:41:23,838 : INFO : sample=0.001 downsamples 43 most-common words
2019-05-31 01:41:23,840 : INFO : downsam

2019-05-31 01:42:26,117 : INFO : EPOCH 2 - PROGRESS: at 86.84% examples, 130578 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:42:27,129 : INFO : EPOCH 2 - PROGRESS: at 89.74% examples, 130254 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:42:28,197 : INFO : EPOCH 2 - PROGRESS: at 93.16% examples, 130678 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:42:29,286 : INFO : EPOCH 2 - PROGRESS: at 97.49% examples, 131897 words/s, in_qsize 4, out_qsize 1
2019-05-31 01:42:30,106 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-31 01:42:30,115 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-31 01:42:30,169 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-31 01:42:30,171 : INFO : EPOCH - 2 : training on 5844680 raw words (4268349 effective words) took 32.5s, 131502 effective words/s
2019-05-31 01:42:31,212 : INFO : EPOCH 3 - PROGRESS: at 3.20% examples, 137592 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:42:32,262 :

2019-05-31 01:43:33,329 : INFO : EPOCH 5 - PROGRESS: at 20.39% examples, 136986 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:43:34,330 : INFO : EPOCH 5 - PROGRESS: at 22.14% examples, 128026 words/s, in_qsize 6, out_qsize 0
2019-05-31 01:43:35,399 : INFO : EPOCH 5 - PROGRESS: at 24.32% examples, 122917 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:43:36,401 : INFO : EPOCH 5 - PROGRESS: at 25.66% examples, 115978 words/s, in_qsize 5, out_qsize 0
2019-05-31 01:43:37,563 : INFO : EPOCH 5 - PROGRESS: at 26.90% examples, 108700 words/s, in_qsize 6, out_qsize 0
2019-05-31 01:43:38,568 : INFO : EPOCH 5 - PROGRESS: at 28.06% examples, 103651 words/s, in_qsize 6, out_qsize 0
2019-05-31 01:43:39,648 : INFO : EPOCH 5 - PROGRESS: at 29.21% examples, 98791 words/s, in_qsize 6, out_qsize 0
2019-05-31 01:43:40,733 : INFO : EPOCH 5 - PROGRESS: at 30.63% examples, 95162 words/s, in_qsize 6, out_qsize 0
2019-05-31 01:43:41,788 : INFO : EPOCH 5 - PROGRESS: at 31.75% examples, 91745 words/s, in_qsize 5

2019-05-31 01:44:56,989 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-31 01:44:57,017 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-31 01:44:57,187 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-31 01:44:57,193 : INFO : EPOCH - 5 : training on 5844680 raw words (4268252 effective words) took 90.3s, 47282 effective words/s
2019-05-31 01:44:57,201 : INFO : training on a 29223400 raw words (21339589 effective words) took 212.5s, 100436 effective words/s


### Load pre-trained embeddings:

In [20]:
# It's for later

#from gensim.test.utils import datapath
#w2v = KeyedVectors.load_word2vec_format(datapath('downloaded_vectors_path'), binary=False)  

In Gensim, embeddings are loaded and can be used via the ["KeyedVectors"](https://radimrehurek.com/gensim/models/keyedvectors.html) class

> Since trained word vectors are independent from the way they were trained (Word2Vec, FastText, WordRank, VarEmbed etc), they can be represented by a standalone structure, as implemented in this module.

>The structure is called “KeyedVectors” and is essentially a mapping between entities and vectors. Each entity is identified by its string id, so this is a mapping between {str => 1D numpy array}.

>The entity typically corresponds to a word (so the mapping maps words to 1D vectors), but for some models, they key can also correspond to a document, a graph node etc. To generalize over different use-cases, this module calls the keys entities. Each entity is always represented by its string id, no matter whether the entity is a word, a document or a graph node.

## STEP 2: Test learnt embeddings

The word embedding space directly encodes similarities between words: the vector coding for the word "great" will be closer to the vector coding for "good" than to the one coding for "bad". Generally, [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) is the distance used when considering distance between vectors.

KeyedVectors have a built in [similarity](https://radimrehurek.com/gensim/models /keyedvectors.html#gensim.models.keyedvectors.BaseKeyedVectors.similarity) method to compute the cosine similarity between words

In [21]:
# is great really closer to good than to bad ?
print("great and good:",w2v.wv.similarity("great","good"))
print("great and bad:",w2v.wv.similarity("great","bad"))

great and good: 0.7742736919409504
great and bad: 0.4786650067084871


Since cosine distance encodes similarity, neighboring words are supposed to be similar. The [most_similar](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.BaseKeyedVectors.most_similar) method returns the `topn` words given a query.

In [22]:
# The query can be as simple as a word, such as "movie"

# Try changing the word
w2v.wv.most_similar("movie",topn=5) # 5 most similar words
#w2v.wv.most_similar("awesome",topn=5)
#w2v.wv.most_similar("actor",topn=5)

2019-05-31 01:46:00,348 : INFO : precomputing L2-norms of word weight vectors


[('film', 0.9409251809120178),
 ('"film"', 0.8615127801895142),
 ('"movie"', 0.7822023630142212),
 ('programme', 0.7682238221168518),
 ('movie,', 0.7540420293807983)]

But it can be a more complicated query
Word embedding spaces tend to encode much more.

The most famous exemple is: `vec(king) - vec(man) + vec(woman) => vec(queen)`

In [23]:
# What is awesome - good + bad ?
w2v.wv.most_similar(positive=["horror","good"],negative=["fear"],topn=3)  


# Try other things like plurals for exemple.


[('B', 0.6892256736755371),
 ('"B"', 0.6766188144683838),
 ('low-budget', 0.6486255526542664)]

To test learnt "synctactic" and "semantic" similarities, Mikolov et al. introduced a special dataset containing a wide variety of three way similarities.

In [24]:
out = w2v.wv.evaluate_word_analogies("data/questions-words.txt",case_insensitive=True)  #original semantic syntactic dataset.

AttributeError: 'Word2VecKeyedVectors' object has no attribute 'evaluate_word_analogies'

When training the w2v models on the review dataset, since it hasn't been learnt with a lot of data, it does not perform very well. 


## STEP 3:  sentiment classification

In the previous practical session, we used a bag of word approach to transform text into vectors.
Here, we propose to try to use word vectors (previously learnt or loaded).


### <font color='green'> Since we have only word vectors and that sentences are made of multiple words, we need to aggregate them. </font>


### (1) Vectorize reviews using word vectors:

Word aggregation can be done in different ways:

- Sum
- Average
- Min/feature
- Max/feature

#### a few pointers:

- `w2v.wv.vocab` is a `set()` of the vocabulary (all existing words in your model)
- `np.minimum(a,b) and np.maximum(a,b)` respectively return element-wise min/max 

In [25]:
import numpy as np
# We first need to vectorize text:
# First we propose to a sum of them

vocab = w2v.wv.vocab
def vectorize(text,mean=False):
    """
    This function should vectorize one review

    input: str
    output: np.array(float)
    """    
    vec = 0 # pfffff
    
    for word in text.split():
        if word in vocab:
            vec += w2v.wv[word]
         
        
        
    return np.array(vec)
    

classes = [pol for text,pol in train]
X = [vectorize(text) for text,pol in train]
X_test = [vectorize(text) for text,pol in test]
true = [pol for text,pol in test]

#let's see what a review vector looks like.
print(X[0])

[-37.71378     24.333488     1.6598583  -10.220578    18.240505
 -15.804233     1.2339287   -1.4720343   -4.8420196   -0.43907475
  19.982637     4.265581    11.705099    -9.296954    -0.73340267
  13.870491    14.8589945   -5.006123     3.3399227   22.8701
  27.991766   -25.142494     4.2121315   14.577418   -12.8555765
   7.963733   -24.56315     28.534693     1.9523832  -15.280476
  -7.911737    -0.6272571   -2.6366193    0.75962913  -1.5832696
   4.734096    11.603198   -39.25363      2.6582382   10.530556
 -18.549051    19.82562      3.1365051   18.462889    16.724194
 -32.188755    10.507522   -13.188044    -5.3750105   -3.6017435
  -2.4678547    1.0381603    7.40374    -10.346022   -29.409735
  20.568565    13.945981   -18.047895     5.9508843   -0.52349937
  18.530285    -2.8346026  -19.846073   -17.214226   -21.067265
   9.249025    32.162525    -0.10045007  18.814463    28.051416
   2.4045773   -5.3584075  -18.295898   -28.165567   -11.051328
   1.2596935  -17.556528    -8.77

### (2) Train a classifier 
as in the previous practical session, train a logistic regression to do sentiment classification with word vectors



In [32]:


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Scikit Logistic Regression
clf = LogisticRegression().fit(X, classes)
Ypred = clf.predict(X_test)
accuracy_score(Ypred,true)

0.82464

performance should be worst than with bag of word (~80%). Sum/Mean aggregation does not work well on long reviews (especially with many frequent words). This adds a lot of noise.

## **Todo** :  Try answering the following questions:

- Which word2vec model works best: skip-gram or cbow
- Do pretrained vectors work best than those learnt on the train dataset ?


Il y a pas une grosse différence entre les deux modèles. Cbow est plus rapide que skip gram mais les résultats sont globalement les mêmes. Cbow prend le contexte sachant les mots et SG l'inverse.


**(Bonus)** To have a better accuracy, we could try two things:
- Better aggregation methods (weight by tf-idf ?)
- Another word vectorizing method such as [fasttext](https://radimrehurek.com/gensim/models/fasttext.html)
- A document vectorizing method such as [Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html)

## --- Generate text with a recurrent neural network (Pytorch) ---
### (Mostly Read & Run)

The goal is to replicate the (famous) experiment from [Karpathy's blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

To learn to generate text, we train a recurrent neural network to do the following task:

Given a "chunk" of text: `this is random text`

the goal of the network is to predict each character in **`his is random text` ** sequentially given the following sequential input **`this is random tex`**:




## Load text (dataset/input.txt)

Before building training batch, we load the full text in RAM

In [33]:
import unidecode
import string
import random
import re
import torch
import torch.nn as nn

all_characters = string.printable
n_characters = len(all_characters)

file = unidecode.unidecode(open('data/input.txt').read()) #clean text => only ascii
file_len = len(file)
print('file_len =', file_len)


file_len = 1115394


## 2: Helper functions:

We have a text and we want to feed batch of chunks to a neural network:

one chunk  A,B,C,D,E
[input] A,B,C,D -> B,C,D,E [output]

Note: we will use an embedding layer instead of a one-hot encoding scheme.

for this, we have 3 functions:

- One to get a random str chunk of size `chunk_len` : `random_chunk` 
- One to turn a chunk into a tensor of size `(1,chunk_len)` coding for each characters : `char_tensor`
- One to return random input and output chunks of size `(batch_size,chunk_len)` : `random_training_set`




In [34]:
import time, math


#Get a piece of text
def random_chunk(chunk_len):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]


# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(1,len(string)).long()
    for c in range(len(string)):
        tensor[0,c] = all_characters.index(string[c])
    return tensor


#Turn a piece of text in train/test
def random_training_set(chunk_len=200, batch_size=8):
    chunks = [random_chunk(chunk_len) for _ in range(batch_size)]
    inp = torch.cat([char_tensor(chunk[:-1]) for chunk in chunks],dim=0)
    target = torch.cat([char_tensor(chunk[1:]) for chunk in chunks],dim=0)
    
    return inp, target

print(random_training_set(10,4))  ## should return 8 chunks of 10 letters. 

(tensor([[94, 17, 14, 94, 13, 24, 29, 17, 94, 28],
        [75, 94, 44, 94, 21, 24, 31, 14, 94, 29],
        [21, 10, 13, 34, 94, 21, 18, 31, 18, 23],
        [22, 18, 23, 18, 28, 29, 14, 27, 96, 56]]), tensor([[17, 14, 94, 13, 24, 29, 17, 94, 28, 14],
        [94, 44, 94, 21, 24, 31, 14, 94, 29, 17],
        [10, 13, 34, 94, 21, 18, 31, 18, 23, 16],
        [18, 23, 18, 28, 29, 14, 27, 96, 56, 23]]))


## The actual RNN model (only thing to complete):

It should be composed of three distinct modules:

- an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) (n_characters, hidden_size)

```
nn.Embedding(len_dic,size_vec)
```
- a [recurrent](https://pytorch.org/docs/stable/nn.html#recurrent-layers) layer (hidden_size, hidden_size)
```
nn.RNN(in_size,out_size) or nn.GRU() or nn.LSTM() => rnn_cell parameter
```
- a [prediction](https://pytorch.org/docs/stable/nn.html#linear) layer (hidden_size, output_size)

```
nn.Linear(in_size,out_size)
```
=> Complete the `init` function code

In [35]:
import torch.nn.functional as f

class RNN(nn.Module):
    
    def __init__(self, n_char, hidden_size, output_size, n_layers=1,rnn_cell=nn.RNN):
        """
        Create the network
        """
        super(RNN, self).__init__()
        
        self.n_char = n_char
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        #  (batch,chunk_len) -> (batch, chunk_len, hidden_size)  
        self.embed = nn.Embedding(n_char,hidden_size)
        
        # (batch, chunk_len, hidden_size)  -> (batch, chunk_len, hidden_size)  
        self.rnn = nn.RNN(hidden_size,hidden_size)
        
        #(batch, chunk_len, hidden_size) -> (batch, chunk_len, output_size)  
        self.predict = nn.Linear(hidden_size,output_size)
    
    def forward(self, input):
        """
        batched forward: input is (batch > 1,chunk_len)
        """
        input = self.embed(input)
        output,_  = self.rnn(input)
        output = self.predict(f.tanh(output))
        return output
    
    def forward_seq(self, input,hidden=None):
        """
        not batched forward: input is  (1,chunk_len)
        """
        input = self.embed(input)
        output,hidden  = self.rnn(input.unsqueeze(0),hidden)
        output = self.predict(f.tanh(output))
        return output,hidden


## Text generation function

Sample text from the model

In [36]:
def generate(model,prime_str='A', predict_len=100, temperature=0.8):
    prime_input = char_tensor(prime_str).squeeze(0)
    hidden = None
    predicted = prime_str+""
    # Use priming string to "build up" hidden state

    for p in range(len(prime_str)-1):
        _,hidden = model.forward_seq(prime_input[p].unsqueeze(0),hidden)
            
    #print(hidden.size())
    for p in range(predict_len):
        output, hidden = model.forward_seq(prime_input[-1].unsqueeze(0), hidden)
                # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        #print(output_dist)
        top_i = torch.multinomial(output_dist, 1)[0]
        #print(top_i)
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        prime_input = torch.cat([prime_input,char_tensor(predicted_char).squeeze(0)])

    return predicted



## Training loop for net

In [39]:
def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

###Parameters
n_epochs = 20000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 5
lr = 0.005
batch_size = 16
chunk_len = 80

####

model = RNN(n_characters, hidden_size, n_characters, n_layers) #create model
model_optimizer = torch.optim.Adam(model.parameters(), lr=lr) #create Adam optimizer
criterion = nn.CrossEntropyLoss() #chose criterion

start = time.time()
all_losses = []
loss_avg = 0


def train(inp, target):
    """
    Train sequence for one chunk:
    """
    #reset gradients
    model_optimizer.zero_grad() 
    
    # predict output
    output = model(inp)
    
    #compute loss
    loss =  criterion(output.view(batch_size*chunk_len,-1), target.view(-1)) 

    #compute gradients and backpropagate
    loss.backward() 
    model_optimizer.step() 

    return loss.data.item() 



for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set(chunk_len,batch_size))  #train on one chunk 
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(generate(model,'Wh', 100), '\n')
       


    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0


KeyboardInterrupt: 

## Visualize loss 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

## Try different temperatures

Changing the distribution sharpness has an impact on character sampling:

more or less probable things are sampled

In [74]:
print(generate(model,'T', 200, temperature=1))
print("----")
print(generate(model,'Th', 200, temperature=0.8))
print("----")

print(generate(model,'Th', 200, temperature=0.5))
print("----")

print(generate(model,'Th', 200, temperature=0.3))
print("----")

print(generate(model,'Th', 200, temperature=0.1))

/usr/local/lib/python3.5/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


This tis s wnd oine wind t'd me serkerpot? dis thoud?
Shexist t it head tnomoookeruronds kirerinorerd mmusthan ck neopauis, ben h s, thaghig ss the
PO:
St,
Wat nyoutowe o perif-deimatoost r f t sthevea
----
Theivin hande he be meameded irof d toulled for ter th

HARI ind sthan s, t t polead s ay ver, ttik hereis, t IO:

ULAnd the meyod bes s ms keasthendy keng or ngres?
S:


LIn hek arir se pl meartt ho tr
----
The there ar h th te,

Son my t re tou y cal m te tow al ber m
Whand an the t ghend s y thald hof t the the hand the the thofour the e the he thar fon that ENo t rmay thind oure thime be t we mave athed
----
The me our t ar n the be the the the the s thand t bere t me t mere me me t the y bend al t t the t be the that thar t the s the t thes thand the thers s pand the t t the the mer mathe the thar t the t 
----
The the t the the the the the t the the the t the the the t the the the the the the the the the the the the the the the the the the the the the the the the the the me th

### Improving this code:

(a) Tinker with parameters:

- Is it really necessary to have 100 dims character embeddings
- Chunk length can be gradually increased
- Try changing RNN cell type (GRUs - LSTMs)

(b) Add GPU support to go faster


## ------ End of practical

#### Legacy loading code

In [77]:
import glob
from os.path import split as pathsplit

dir_train = "dataset/aclImdb/train/"
dir_test = "dataset/aclImdb/test/"

train_files = glob.glob(dir_train+'pos/*.txt') + glob.glob(dir_train+'neg/*.txt')
test_files = glob.glob(dir_test+'pos/*.txt') + glob.glob(dir_test+'neg/*.txt')


def get_polarity(f):
    """
    Extracts polarity from filename:
    0 is negative (< 5)
    1 is positive (> 5)
    """
    _,name = pathsplit(f)
    if int(name.split('_')[1].split('.')[0]) < 5:
        return 0
    else:
        return 1


def open_one(f):
    
    polarity = get_polarity(f)
    
    with open(f,"r") as review:
        text = " ".join(review.readlines()).strip()
    
    return (text,polarity)

print(open_one(train_files[0]))

train = [open_one(x) for x in train_files] #contains (text,pol) couples
test = [open_one(x) for x in test_files]   #contains (text,pol) couples


0


IndexError: list index out of range